In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr  4 08:59:33 2021

@author: forustous
"""
#%%
SEED = 0

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
from sklearn.utils.class_weight import compute_sample_weight

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
df=pd.read_csv("/home/danssa/proj_ua/data/60model_train_share.csv")

X_train = df.drop('eGFR_ab', axis=1)
y_train = df['eGFR_ab'].astype("int64")

weights = compute_sample_weight(class_weight="balanced", y=y_train)

In [11]:
#%% 
# optuna

automl = AutoML(mode="Optuna", ml_task="binary_classification", 
    algorithms=["CatBoost"], eval_metric='auc',
    optuna_time_budget=10*60,
    total_time_limit = 24*3600,
    golden_features = False, 
    features_selection = False,
    train_ensemble= True,
    stack_models = 'auto',
    random_state=SEED, results_path="optuna")

automl.fit(X_train, y_train, weights)

This model has already been fitted. You can use predict methods or select a new 'results_path' for a new 'fit()'.


In [6]:
automl.report()

In [7]:
# %%
# explain

automl = AutoML(mode="Explain", ml_task="binary_classification", 
    algorithms=["Baseline", "CatBoost", "Xgboost", "Random Forest", "Extra Trees", "LightGBM", "Neural Network"], 
    eval_metric='auc',
    train_ensemble= False, 
    random_state=SEED,
    results_path="explain-wt")

automl.fit(X_train, y_train, weights)

AutoML directory: explain-wt
The task is binary_classification with evaluation metric auc
AutoML will use algorithms: ['Baseline', 'CatBoost', 'Xgboost', 'Random Forest', 'Extra Trees', 'LightGBM', 'Neural Network']
AutoML steps: ['simple_algorithms', 'default_algorithms']
* Step simple_algorithms will try to check up to 1 model
1_Baseline auc 0.5 trained in 0.59 seconds
* Step default_algorithms will try to check up to 6 models
C extension was not built during install!
2_Default_LightGBM auc 0.923095 trained in 166.17 seconds
C extension was not built during install!
Exception while producing SHAP explanations. numpy.core.multiarray failed to import
Continuing ...
3_Default_Xgboost auc 0.923605 trained in 18.65 seconds
4_Default_CatBoost auc 0.924505 trained in 19.16 seconds
5_Default_NeuralNetwork auc 0.919984 trained in 47.91 seconds
C extension was not built during install!
Exception while producing SHAP explanations. numpy.core.multiarray failed to import
Continuing ...
6_Default_

AutoML(algorithms=['Baseline', 'CatBoost', 'Xgboost', 'Random Forest',
                   'Extra Trees', 'LightGBM', 'Neural Network'],
       eval_metric='auc', ml_task='binary_classification', random_state=0,
       results_path='explain-wt', train_ensemble=False)

In [9]:
automl.report()

In [10]:
# %%
# Perform

automl = AutoML(mode="Perform", ml_task="binary_classification", 
    algorithms=["CatBoost", "Xgboost"], 
    eval_metric='auc',
    golden_features=False,
    features_selection=False,
    train_ensemble = False,
    stack_models = False, 
    random_state=SEED,
    results_path="perform")

automl.fit(X_train, y_train, weights)

# %%

AutoML directory: perform
The task is binary_classification with evaluation metric auc
AutoML will use algorithms: ['CatBoost', 'Xgboost']
AutoML steps: ['simple_algorithms', 'default_algorithms', 'not_so_random', 'hill_climbing_1', 'hill_climbing_2']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 2 models
1_Default_Xgboost auc 0.923201 trained in 100.16 seconds (1-sample predict time 0.0381 seconds)
2_Default_CatBoost auc 0.923681 trained in 85.53 seconds (1-sample predict time 0.0638 seconds)
* Step not_so_random will try to check up to 8 models
3_Xgboost auc 0.922543 trained in 71.28 seconds (1-sample predict time 0.0377 seconds)
7_CatBoost auc 0.922913 trained in 62.75 seconds (1-sample predict time 0.0612 seconds)
4_Xgboost auc 0.921557 trained in 75.29 seconds (1-sample predict time 0.0388 seconds)
8_CatBoost auc 0.923144 trained in 69.6 seconds (1-sample predict time 0.048 seconds)
5_Xgboost auc 0.923039 trained in 

AutoML(algorithms=['CatBoost', 'Xgboost'], eval_metric='auc',
       features_selection=False, golden_features=False,
       ml_task='binary_classification', mode='Perform', random_state=0,
       results_path='perform', stack_models=False, train_ensemble=False)